### Load the data

In [32]:
import numpy as np
import pandas as pd

table = pd.read_csv("positionData.csv", low_memory=False)
display(table.head())

,TOPchampId,TOPkeyRune,TOPspell1,TOPspell2,JNGchampId,JNGkeyRune,JNGspell1,JNGspell2,MIDchampId,MIDkeyRune,MIDspell1,MIDspell2,ADCchampId,ADCkeyRune,ADCspell1,ADCspell2,SUPchampId,SUPkeyRune,SUPspell1,SUPspell2
0,19,2,4,5,56,4,4,9,18,3,4,0,118,0,4,7,34,14,4,5
1,0,13,4,0,54,4,4,9,139,7,4,5,47,0,4,7,10,4,4,5
2,43,2,4,0,36,5,6,9,131,8,4,0,64,0,4,0,74,7,4,5
3,93,2,4,0,42,4,4,9,29,4,4,5,21,2,4,7,12,8,4,5
4,58,2,4,0,79,6,4,9,117,8,4,5,45,3,4,7,140,8,4,5


### Prepare the data extracting x and y

In [33]:
import time
import itertools

b = time.time()

perms = np.array(list(itertools.permutations([0, 1, 2, 3, 4])))
y_cac = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]])

x = np.zeros((len(table)//5*120, 20))
y = np.zeros((len(table)//5*120, 25))

for i in range(len(table)//5):
    x_cac = np.array([table.iloc[i,0:4], table.iloc[i,4:8], table.iloc[i,8:12], table.iloc[i,12:16], table.iloc[i,16:20]])
         
    for perm in range(len(perms)):
        
        for e in range(len(perms[perm])):
            x[i*120+perm][4*e:4*(e+1)] = x_cac[perms[perm][e]]
            y[i*120+perm][5*e:5*(e+1)] = y_cac[perms[perm][e]]

print("Execution time:", round(time.time()-b, 2), "seconds")

Execution time: 18.24 seconds


### Shuffle the data and check it is ok

In [39]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

x, y = unison_shuffled_copies(x, y)

print(x[2])
print(y[2])

[  2.   4.   4.   0.  12.   8.   4.   0. 136.   4.   4.   5.  96.  14.
   4.   9.   3.  14.   4.   5.]
[1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1.]


### Create the loaders and separate training and testing data

In [40]:
import torch
import torch.utils.data

train_ratio = 0.8
bs = 64

thold = int(len(x)*train_ratio)

train = list(zip(x[:thold], y[:thold]))
trainloader = torch.utils.data.DataLoader(train, batch_size=bs, shuffle=True, num_workers=2)

test = list(zip(x[thold:], y[thold:]))
testloader = torch.utils.data.DataLoader(test, batch_size=bs, shuffle=True, num_workers=2)

### Define the Neural Network

In [48]:
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0")


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        """
        self.idC1 = nn.Linear(141, 50)
        self.runeC1 = nn.Linear(17, 9)
        self.spellsC1 = nn.Linear(10, 5)
        
        self.idC2 = nn.Linear(141, 50)
        self.runeC2 = nn.Linear(17, 9)
        self.spellsC2 = nn.Linear(10, 5)
        
        self.idC3 = nn.Linear(141, 50)
        self.runeC3 = nn.Linear(17, 9)
        self.spellsC3 = nn.Linear(10, 5)
        
        self.idC4 = nn.Linear(141, 50)
        self.runeC4 = nn.Linear(17, 9)
        self.spellsC4 = nn.Linear(10, 5)
        
        self.idC5 = nn.Linear(141, 50)
        self.runeC5 = nn.Linear(17, 9)
        self.spellsC5 = nn.Linear(10, 5)
        """
        
        self.fc2 = nn.Linear((50+9+5)*5, 500)
        self.fc3 = nn.Linear(500, 25)
        
        
        self.idC = nn.Linear(141, 50)
        self.runeC = nn.Linear(17, 9)
        self.spellsC = nn.Linear(10, 5)
        
        self.id = torch.zeros(5, bs, 141)#.to(device)
        self.rune = torch.zeros(5, bs, 17)#.to(device)
        self.spells = torch.zeros(5, bs, 10)#.to(device)
        

    def forward(self, x):
        self.id.fill_(0)
        self.rune.fill_(0)
        self.spells.fill_(0)
        
        for j in range(5):
            for i in range(len(x)):
                self.id[j][i][int(x[i][j*4+0].item())] = 1
                self.rune[j][i][int(x[i][j*4+1].item())] = 1
                self.spells[j][i][int(x[i][j*4+2].item())] = 1
                self.spells[j][i][int(x[i][j*4+3].item())] = 1
        
        x = torch.cat(
            (
            F.relu(self.idC(self.id[0])), F.relu(self.runeC(self.rune[0])), F.relu(self.spellsC(self.spells[0])),
            F.relu(self.idC(self.id[1])), F.relu(self.runeC(self.rune[1])), F.relu(self.spellsC(self.spells[1])),
            F.relu(self.idC(self.id[2])), F.relu(self.runeC(self.rune[2])), F.relu(self.spellsC(self.spells[2])),
            F.relu(self.idC(self.id[3])), F.relu(self.runeC(self.rune[3])), F.relu(self.spellsC(self.spells[3])),
            F.relu(self.idC(self.id[4])), F.relu(self.runeC(self.rune[4])), F.relu(self.spellsC(self.spells[4]))
            ), dim=1)
        
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))

        return x

net = Net()
#net.to(device)

### Define a loss function and a optimizer

In [53]:
import torch.optim as optim

#criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

### Train the network

In [54]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0 # mean loss in the 2000 batches between printing and printing
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = F.binary_cross_entropy(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        if i % 1000 == 999:
            lossc = 0
            count = 0
            for data in testloader:
                images, labels = data
                #images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                lossc += F.binary_cross_entropy(outputs, labels.float()).item()
                if count > 10:
                    break
                count += 1
            print('test loss: '+str(lossc/count))

print('Finished Training')

D:\ANACONDA\lib\site-packages\torch\nn\functional.py:1025: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   100] loss: 0.496
[1,   200] loss: 0.440
[1,   300] loss: 0.323
[1,   400] loss: 0.292
[1,   500] loss: 0.271
[1,   600] loss: 0.244
[1,   700] loss: 0.218
[1,   800] loss: 0.197
[1,   900] loss: 0.175
[1,  1000] loss: 0.156
loss: 0.16063394262032074
[2,   100] loss: 0.135
[2,   200] loss: 0.119
[2,   300] loss: 0.106
[2,   400] loss: 0.098
[2,   500] loss: 0.096
[2,   600] loss: 0.091
[2,   700] loss: 0.085
[2,   800] loss: 0.084
[2,   900] loss: 0.082
[2,  1000] loss: 0.080
loss: 0.08726714348251169
[3,   100] loss: 0.076
[3,   200] loss: 0.075
[3,   300] loss: 0.073
[3,   400] loss: 0.075
[3,   500] loss: 0.072
[3,   600] loss: 0.070
[3,   700] loss: 0.071
[3,   800] loss: 0.068
[3,   900] loss: 0.070
[3,  1000] loss: 0.068
loss: 0.07659323073246262


KeyboardInterrupt: 

In [55]:
lossc = 0
count = 0
for data in testloader:
    images, labels = data
    #images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    print(outputs[0])
    lossc += F.binary_cross_entropy(outputs, labels.float()).item()
    if count > 100:
        break
    count += 1
print('loss: '+str(lossc/count))

Traceback (most recent call last):
  File "D:\ANACONDA\lib\multiprocessing\queues.py", line 230, in _feed
    close()
  File "D:\ANACONDA\lib\multiprocessing\connection.py", line 177, in close
    self._close()
  File "D:\ANACONDA\lib\multiprocessing\connection.py", line 277, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] Controlador no válido
Traceback (most recent call last):
  File "D:\ANACONDA\lib\multiprocessing\queues.py", line 230, in _feed
    close()
  File "D:\ANACONDA\lib\multiprocessing\connection.py", line 177, in close
    self._close()
  File "D:\ANACONDA\lib\multiprocessing\connection.py", line 277, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] Controlador no válido
D:\ANACONDA\lib\site-packages\torch\nn\functional.py:1025: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([2.6716e-05, 9.9546e-01, 8.9876e-04, 9.6878e-04, 2.2246e-04, 1.2796e-02,
        6.7282e-04, 7.3460e-02, 9.8236e-02, 2.8474e-01, 4.2534e-02, 1.4140e-04,
        4.9287e-01, 3.6340e-01, 2.5552e-04, 8.8882e-01, 1.4658e-04, 2.6852e-02,
        4.2091e-01, 3.6230e-06, 1.5913e-02, 1.2935e-03, 4.5858e-03, 9.4464e-04,
        9.9649e-01], grad_fn=<SelectBackward>)
tensor([3.4826e-03, 1.1018e-03, 1.3422e-03, 6.8917e-05, 9.9984e-01, 3.0487e-02,
        7.2432e-05, 8.7914e-01, 2.3751e-02, 1.8623e-03, 3.3981e-04, 9.8887e-01,
        1.3270e-02, 2.1401e-05, 1.8556e-05, 9.6178e-01, 7.1537e-05, 1.6737e-01,
        1.1352e-02, 2.2884e-06, 2.8691e-04, 2.0733e-03, 6.0029e-03, 9.8732e-01,
        1.7064e-03], grad_fn=<SelectBackward>)
tensor([0.9589, 0.0001, 0.0002, 0.0356, 0.0306, 0.0016, 0.9978, 0.0003, 0.0015,
        0.0000, 0.0069, 0.0001, 0.9955, 0.0086, 0.0047, 0.0020, 0.0011, 0.0119,
        0.0816, 0.8716, 0.0284, 0.0014, 0.0250, 0.7097, 0.0025],
       grad_fn=<SelectBackward>)
tensor([

       grad_fn=<SelectBackward>)
tensor([7.6086e-03, 5.8098e-05, 9.8855e-01, 3.9200e-04, 4.9919e-03, 2.7131e-04,
        9.9147e-01, 8.5347e-04, 1.2532e-04, 2.2059e-04, 5.3913e-04, 9.2730e-04,
        2.5721e-03, 9.9574e-01, 2.4521e-04, 9.7080e-01, 6.2231e-05, 1.0593e-01,
        7.1844e-03, 4.8806e-06, 1.2470e-02, 7.3465e-04, 2.0700e-03, 4.5653e-05,
        9.9977e-01], grad_fn=<SelectBackward>)
tensor([0.0064, 0.0001, 0.9966, 0.0043, 0.0012, 0.0117, 0.0002, 0.0105, 0.9731,
        0.0017, 0.9633, 0.0000, 0.0026, 0.0267, 0.0005, 0.0008, 0.0016, 0.0040,
        0.0060, 0.9938, 0.0001, 0.9982, 0.0012, 0.0007, 0.0001],
       grad_fn=<SelectBackward>)
tensor([9.2124e-04, 1.2201e-03, 1.5282e-02, 2.1921e-01, 7.4780e-01, 5.7763e-03,
        1.5036e-03, 8.9926e-04, 6.8621e-02, 9.7432e-01, 4.3268e-04, 9.9468e-01,
        8.7605e-04, 2.4246e-02, 1.5594e-06, 4.3490e-03, 1.2431e-04, 9.9855e-01,
        5.3965e-02, 1.7253e-04, 9.9894e-01, 1.6864e-04, 3.4037e-04, 2.4775e-02,
        1.8440e-05], g

       grad_fn=<SelectBackward>)
tensor([0.9916, 0.0000, 0.0091, 0.0004, 0.0007, 0.0001, 0.9920, 0.0031, 0.0001,
        0.0006, 0.0001, 0.0020, 0.0046, 0.9983, 0.0012, 0.0404, 0.0000, 0.9987,
        0.0070, 0.0000, 0.0058, 0.0020, 0.0021, 0.0004, 0.9986],
       grad_fn=<SelectBackward>)
tensor([9.9423e-01, 9.4190e-06, 1.7152e-02, 3.1862e-03, 2.7014e-04, 2.4876e-03,
        4.2839e-01, 9.2652e-01, 2.7434e-04, 8.0300e-06, 7.2735e-05, 3.9938e-04,
        1.7223e-02, 9.9967e-01, 4.8323e-03, 6.7676e-04, 7.4896e-01, 2.4941e-02,
        9.2844e-05, 3.3105e-03, 3.2413e-02, 1.8179e-04, 3.9087e-02, 1.0204e-03,
        9.9826e-01], grad_fn=<SelectBackward>)
tensor([0.0021, 0.4464, 0.7273, 0.0001, 0.0001, 0.0055, 0.0003, 0.0117, 0.9864,
        0.0486, 0.0013, 0.9128, 0.0024, 0.0001, 0.0179, 0.0455, 0.0001, 0.1283,
        0.0027, 0.9730, 0.8872, 0.0000, 0.0415, 0.3091, 0.0006],
       grad_fn=<SelectBackward>)
tensor([0.0005, 0.9899, 0.0028, 0.0000, 0.0001, 0.0002, 0.0024, 0.0058, 0.9971,
    

       grad_fn=<SelectBackward>)
tensor([5.0895e-05, 1.6008e-03, 6.7243e-02, 1.2578e-03, 9.9646e-01, 8.8578e-01,
        8.8028e-05, 1.6627e-01, 1.0245e-02, 9.1122e-05, 6.4802e-04, 2.4935e-03,
        2.3511e-03, 9.7896e-01, 6.6254e-03, 9.0155e-04, 9.9105e-01, 7.4082e-04,
        5.7236e-06, 2.8890e-03, 2.2756e-01, 4.4615e-05, 9.6541e-01, 3.0022e-03,
        2.7738e-04], grad_fn=<SelectBackward>)
tensor([1.3058e-02, 9.6848e-04, 3.2221e-03, 2.0891e-03, 9.9512e-01, 1.3395e-03,
        4.2665e-04, 6.8583e-01, 4.8360e-01, 1.2249e-02, 6.6935e-01, 1.8348e-04,
        2.3404e-03, 9.1847e-01, 5.9466e-06, 6.6510e-01, 1.4944e-04, 3.3170e-01,
        3.6903e-03, 9.4625e-03, 1.9142e-04, 9.9849e-01, 8.5124e-04, 5.7063e-04,
        1.7633e-04], grad_fn=<SelectBackward>)
tensor([1.7301e-03, 1.0727e-03, 9.3800e-03, 1.0302e-04, 9.9876e-01, 4.6756e-02,
        1.2794e-04, 9.3018e-01, 7.2109e-03, 5.9304e-04, 8.7009e-05, 1.5645e-03,
        5.4066e-03, 9.9882e-01, 4.0618e-04, 9.9302e-01, 6.5036e-05, 1.421

In [13]:
len(testloader)

3558